In [1]:
%pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
%pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
%pip install umap-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import os, json, math, random, pathlib, textwrap, itertools
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import collections
from collections import defaultdict
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.cluster import (KMeans, DBSCAN, Birch, AgglomerativeClustering,
                             OPTICS, SpectralClustering)

from umap import UMAP
import hdbscan
import matplotlib.pyplot as plt
from sklearn.metrics import calinski_harabasz_score
from hdbscan.validity import validity_index
import torch
from transformers import AutoModel, AutoTokenizer

import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-05-17 22:34:53.026638: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-17 22:35:01.273072: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
DATA_PATH = "combined_logs_with_labels.csv"
df = pd.read_csv(DATA_PATH)
df['id'] = df.index

import pandas as pd
import re
from typing import Optional

MIN_LENGTH = 20
MIN_LETTERS = 10
MIN_PRINTABLE_RATIO = 0.7
MAX_SPECIAL_RATIO = 0.1

def is_valid_error_line(line: str) -> bool:
    line = line.strip()
    if len(line) < MIN_LENGTH:
        return False

    letter_count = sum(c.isalpha() for c in line)
    if letter_count < MIN_LETTERS:
        return False

    printable_count = sum(c.isprintable() for c in line)
    if printable_count / len(line) < MIN_PRINTABLE_RATIO:
        return False

    special_chars = re.findall(r'[^\w\s\.\-_:;,?!@#\$%^&*()+=]', line)
    if len(special_chars) / len(line) > MAX_SPECIAL_RATIO:
        return False

    if re.search(r'(\\x[0-9a-fA-F]{2}|�|\u0000|\uFFFD)', line):
        return False

    return True

def process_errors(input_path: str, output_path: Optional[str] = None) -> pd.DataFrame:
    df = pd.read_csv(input_path)
    df['id'] = df.index
    processed_data = []
    for idx, row in df.iterrows():
        if pd.notna(row['errors']):
            for error_line in row['errors'].split('\n'):
                line = error_line.strip()
                if is_valid_error_line(line):
                    processed_data.append({
                        'log_id': idx,
                        'error_text': line
                    })

    result_df = pd.DataFrame(processed_data)
    if output_path:
        result_df.to_csv(output_path, index=False)
    return result_df

PROCESSED_PATH = "processed_errors_clean.csv"
process_errors(DATA_PATH, PROCESSED_PATH)

,log_id,error_text
0,0,Building for target x86_64
1,0,Building for target x86_64
2,0,Hunk #1 succeeded at 129 (offset 12 lines).
3,0,Hunk #2 succeeded at 166 with fuzz 1 (offset 1...
4,0,Hunk #3 succeeded at 299 with fuzz 1 (offset 1...
...,...,...
49593,321,error: Bad exit status from /usr/src/tmp/rpm-t...
49594,321,Bad exit status from /usr/src/tmp/rpm-tmp.7673...
49595,321,Command exited with non-zero status 1
49596,321,hsh-rebuild: rebuild of `libprojectM-2.1.0-alt...


In [7]:
processed_df = pd.read_csv(PROCESSED_PATH)
errors_texts = processed_df['error_text'].astype(str).tolist()

In [8]:
EMB_DIR = pathlib.Path("embeddings"); EMB_DIR.mkdir(exist_ok=True)

MODELS = {
    "distil-log": "teoogherghi/Log-Analysis-Model-DistilBert",
    "roberta-large": "roberta-large",
    "deberta-v3-large": "microsoft/deberta-v3-large",
    "longformer": "allenai/longformer-base-4096"
}

In [9]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH = 32

In [10]:
def encode_st(model_name: str, texts, batch_size=BATCH):
    encoder = SentenceTransformer(model_name, device=DEVICE)
    emb = encoder.encode(texts, batch_size=batch_size,
                         show_progress_bar=True,
                         convert_to_numpy=True, normalize_embeddings=True)
    return emb.astype(np.float32)

In [11]:
from transformers import (
    AutoTokenizer,
    AutoModel,
    RobertaTokenizer,
    DebertaV2Tokenizer
)

In [12]:
for key, hub_id in MODELS.items():
    print(f"⏳ {key}")
    try:
        if key == "distil-log":
            from transformers import DistilBertModel, DistilBertTokenizer
            tok = DistilBertTokenizer.from_pretrained(hub_id)
            mdl = DistilBertModel.from_pretrained(hub_id).to(DEVICE).eval()
            tokenizer_kwargs = {
                'truncation': True,
                'max_length': 256,
                'padding': 'max_length',
                'return_tensors': "pt"
            }

        elif key == "longformer":
            tok = AutoTokenizer.from_pretrained(hub_id)
            mdl = AutoModel.from_pretrained(hub_id, attention_window=256).to(DEVICE).eval()
            tokenizer_kwargs = {
                'truncation': True,
                'max_length': 1024,
                'padding': 'max_length',
                'return_tensors': "pt"
            }

        elif key == "deberta-v3-large":
            tok = DebertaV2Tokenizer.from_pretrained(hub_id)
            mdl = AutoModel.from_pretrained(hub_id).to(DEVICE).eval()
            tokenizer_kwargs = {
                'truncation': True,
                'max_length': 128,
                'padding': 'max_length',
                'return_tensors': "pt"
            }

        elif key == "roberta-large":
            tok = RobertaTokenizer.from_pretrained(hub_id)
            mdl = AutoModel.from_pretrained(hub_id).to(DEVICE).eval()
            tokenizer_kwargs = {
                'truncation': True,
                'max_length': 128,
                'padding': True,
                'return_tensors': "pt"
            }

        else:
            emb = encode_st(hub_id, errors_texts)
            #pd.DataFrame(emb, index=df.id).to_csv(EMB_DIR / f"{key}.csv", index_label="id")
            pd.DataFrame(emb, index=processed_df.log_id).to_csv(EMB_DIR / f"{key}.csv", index_label="id")
            continue

        all_vecs = []
        for i in tqdm(range(0, len(errors_texts), BATCH)):
            batch = errors_texts[i:i+BATCH]
            inputs = tok(batch, **tokenizer_kwargs).to(DEVICE)
            with torch.no_grad():
                outs = mdl(**inputs).last_hidden_state.mean(1)
            all_vecs.append(outs.cpu())

        emb = torch.cat(all_vecs, 0).numpy()
        emb = normalize(emb)
        #pd.DataFrame(emb, index=df.id).to_csv(EMB_DIR / f"{key}.csv", index_label="id")
        pd.DataFrame(emb, index=processed_df.log_id).to_csv(EMB_DIR / f"{key}.csv", index_label="id")

    except Exception as e:
        print(f"⚠️ Critical error with {key}: {str(e)}")
        continue

⏳ distil-log


100%|██████████| 1550/1550 [02:09<00:00, 11.96it/s]


⏳ roberta-large


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1550/1550 [02:19<00:00, 11.14it/s]


⏳ deberta-v3-large


100%|██████████| 1550/1550 [07:28<00:00,  3.46it/s]


⏳ longformer


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
100%|██████████| 1550/1550 [19:19<00:00,  1.34it/s]


In [13]:
CLUSTER_DIR = pathlib.Path("clusters"); CLUSTER_DIR.mkdir(exist_ok=True)


In [14]:
ALGORITHMS = {
    "kmeans": {
        "func": lambda X, params: KMeans(**params).fit(X).labels_,
        "params": [
            {"n_clusters": 20, "n_init": "auto"},
            {"n_clusters": 30, "n_init": 10},
            {"n_clusters": 5, "n_init": 10}
        ]
    },
    "agglo": {
        "func": lambda X, params: AgglomerativeClustering(**params).fit_predict(X),
        "params": [
            {"n_clusters": 20, "linkage": "ward"},
            {"n_clusters": 15, "linkage": "complete"}
        ]
    },
    "dbscan": {
        "func": lambda X, params: DBSCAN(**params).fit_predict(X),
        "params": [
            {"eps": 0.3, "min_samples": 10, "metric": "euclidean"},
            {"eps": 0.5, "min_samples": 15, "metric": "cosine"}
        ]
    },
    "hdbscan": {
        "func": lambda X, params: hdbscan.HDBSCAN(**params).fit_predict(X),
        "params": [
            {"min_cluster_size": 15, "cluster_selection_epsilon": 0.3},
            {"min_cluster_size": 10, "cluster_selection_method": "leaf"},
            {"min_cluster_size": 20, "min_samples": 5}
        ]
    },
    "umap_hdbscan": {
        "func": lambda X, params: hdbscan.HDBSCAN(**params).fit_predict(
            UMAP(n_components=50, random_state=42).fit_transform(X)),
        "params": [{"min_cluster_size": 10}]
    }
}

In [ ]:
def cluster_and_report(name, X, algo_key, params):
    algo_func = ALGORITHMS[algo_key]["func"]
    try:
        labels = algo_func(X, params)
        n_clusters = len(set(labels))

        stats = {
            "n_clusters": n_clusters,
            "noise_ratio": np.mean(labels == -1),
            "silhouette": silhouette_score(X, labels) if n_clusters > 1 else np.nan,
            "davies_bouldin": davies_bouldin_score(X, labels),
            "calinski_harabasz": calinski_harabasz_score(X, labels),
            "cluster_sizes": np.bincount(labels[labels != -1]).tolist()
        }

        if algo_key.startswith("hdbscan"):
            stats["DBCV"] = validity_index(X, labels)

        #label_series = pd.Series(labels, index=df.id, name="cluster")
        label_series = pd.Series(labels, index=processed_df.log_id, name="cluster")
        label_series.to_csv(CLUSTER_DIR / f"{name}_{algo_key}_{param_str(params)}_labels.csv", index_label="id")

        sample_df = pd.DataFrame({'text': df['errors'], 'cluster': labels})
        sample_df.groupby('cluster').apply(lambda x: x.sample(3)).to_csv(
        CLUSTER_DIR / f"{name}_samples.csv")

        return {**stats, "params": params}
    except Exception as e:
        print(f"⚠️ {algo_key} failed: {e}")
        return None

def param_str(params):
    return "_".join(f"{k}={v}" for k,v in params.items())

# %%
results = []

for emb_file in sorted(EMB_DIR.glob("*.csv")):
    name = emb_file.stem
    X = pd.read_csv(emb_file, index_col="id").values
    print(f"\n=== {name} ===")

    for algo_key in ALGORITHMS:
        for params in ALGORITHMS[algo_key]["params"]:
            result = cluster_and_report(name, X, algo_key, params)
            if result:
                results.append({
                    "embedding": name,
                    "algorithm": algo_key,
                    **result
                })
                print(f"{algo_key:8s} [params: {params}] → clusters={result['n_clusters']}")

# %%
CENTRAL_DIR = pathlib.Path("cluster_objects"); CENTRAL_DIR.mkdir(exist_ok=True)

def central_extremes(X, labels, top_k=10):
    out = defaultdict(lambda: {"central": [], "extreme": []})
    for c in set(labels):
        idx = np.where(labels == c)[0]
        part = X[idx]
        center = part.mean(0, keepdims=True)
        dists = np.linalg.norm(part - center, axis=1)
        order = np.argsort(dists)
        out[c]["central"] = idx[order[:top_k]].tolist()
        out[c]["extreme"] = idx[order[::-1][:top_k]].tolist()
    return out

# %%
for lab_file in sorted(CLUSTER_DIR.glob("*_labels.csv")):
    parts = lab_file.stem.split("_")
    name, algo = parts[0], "_".join(parts[1:-1])
    labels = pd.read_csv(lab_file, index_col="id").squeeze().values
    X = pd.read_csv(EMB_DIR / f"{name}.csv", index_col="id").values
    ce = central_extremes(X, labels)

    md_path = CENTRAL_DIR / f"{lab_file.stem}.md"
    with open(md_path, "w", encoding="utf-8") as fp:
        fp.write(f"# {name} + {algo}\n\n")
        fp.write(f"## Cluster Statistics\n")
        fp.write(f"- Total clusters: {len(ce)}\n")
        fp.write(f"- Noise points: {np.sum(labels == -1)}\n\n")

        for c, data in ce.items():
            fp.write(f"## Cluster {c} (Size: {len(data['central']) + len(data['extreme'])})\n\n")
            fp.write("### Central objects\n")
            for ix in data["central"]:
                row = df.loc[ix]
                fp.write(f"* **id={ix}** — `{row.errors}`\n")  # Полный текст
            fp.write("\n### Extreme objects\n")
            for ix in data["extreme"]:
                row = df.loc[ix]
                fp.write(f"* **id={ix}** — `{row.errors}`\n")  # Полный текст
            fp.write("\n---\n")
    print("✓", md_path)

# %%
VIS_DIR = pathlib.Path("viz"); VIS_DIR.mkdir(exist_ok=True)

def umap_plot(name, algo, X, labels):
    reducer = umap.UMAP(n_components=2, random_state=42,
                       n_neighbors=15, min_dist=0.1, metric='cosine')
    emb2d = reducer.fit_transform(X)
    plt.figure(figsize=(10,8))
    scatter = plt.scatter(emb2d[:,0], emb2d[:,1], s=10, c=labels,
                         cmap='tab20', alpha=0.8)
    plt.title(f"{name} + {algo}\nClusters: {len(set(labels))}")
    plt.colorbar(scatter)
    path = VIS_DIR / f"{name}_{algo}.png"
    plt.tight_layout(); plt.savefig(path, dpi=150); plt.close()
    return path
def plot_cluster_distribution(labels):
    plt.figure(figsize=(12,6))
    sizes = np.bincount(labels[labels != -1])
    plt.bar(range(len(sizes)), sizes)
    plt.title(f"Cluster Size Distribution (Total: {len(sizes)})")
    plt.xlabel("Cluster ID")
    plt.ylabel("Number of Samples")
    plt.savefig(VIS_DIR / "cluster_distribution.png")
    plt.close()
# %%
res_df = pd.DataFrame(results)
res_df = res_df.explode("params").reset_index(drop=True)
res_df.to_markdown("full_report.md", index=False)
res_df.to_csv("full_report.csv", index=False)
res_df.head(10)


=== deberta-v3-large ===
⚠️ kmeans failed: array length 49598 does not match index length 2140
⚠️ kmeans failed: array length 49598 does not match index length 2140
⚠️ kmeans failed: array length 49598 does not match index length 2140
⚠️ agglo failed: array length 49598 does not match index length 2140
